First, let's import python modules that we'll need, everything is in
the standard library except for the astpretty module.  We're using
that to output an AST in a more readible format.  Feel free to replace
the ast.pprint calls with ast.dump calls if you want to avoid 
installing the astpretty module 






In [1]:
import argparse
import ast
import collections
import copy
import sys
import timeit

import astpretty

Next we'll define a helper function to time our code fragments. The `timeit.timeit` call runs our code fragment 10,000 times and outputs the time it took to run.  Since timeit runs the code in a clean namespace we need to use the globals keyword arg to pass in the AST that we want to time and then use the `compile` and `exec` calls to first compile an `ast.AST` object to executable code and then to execute it.

In [2]:
def time_tree(tree: ast.AST) -> float:
  """
  Time ast execution

  :param tree: AST tree
  :return: float recording number of seconds to run snippet
  """
  return timeit.timeit(stmt="exec(compile(tree, 'test', mode='exec'))", number=10000, globals={'tree': tree})

We'll use `example.py` as the source code that we'll examine and modify.  The `ast.parse` method will take a string with python code and return a corresponding `ast.AST` object.  We'll use this throughout the code to load and generate an AST from `example.py`

Now, let's look at using the `ast.NodeVisitor` to examine our source code.  First, we'll do the functional equivalent of a hello world program by printing out any functions that we define in our source code.  We use `ast.NodeVisitor` to do this by inheriting from the class and overriding the various visit_* methods.  I've done this in two different ways:

* override `generic_visit` and then if the node we're visiting is an `ast.FunctionDef` object, print out it's name
* override `visit_FunctionDef` and then print out the node name since this method only gets called on FunctionDef nodes

In [3]:
class HelloVisitor(ast.NodeVisitor):

  def generic_visit(self, node):
    if node.__class__ == ast.FunctionDef:
      sys.stdout.write(f"Defining function: {node.name} \n")
    super().generic_visit(node)


class FunctionVisitor(ast.NodeVisitor):

  def visit_FunctionDef(self, node):
    sys.stdout.write(f"Defining function: {node.name} \n")

Finally we'll instantiate our objects and call the `visit` methods on the objects to traverse the AST tree:

In [5]:
  with open("example.py", "r") as source:
    tree = ast.parse(source.read())
  sys.stdout.write("Visiting nodes using generic_visit:\n")
  hello_visitor = HelloVisitor()
  hello_visitor.visit(tree)
  sys.stdout.write("Visiting nodes using specific visit function:\n")
  function_visitor = FunctionVisitor()
  function_visitor.visit(tree)


Visiting nodes using generic_visit:
Defining function: test_function 
Defining function: test_function2 
Visiting nodes using specific visit function:
Defining function: test_function 
Defining function: test_function2 


Now we'll do something a bit more complex with the `NodeVisitor` class. We'll do a bit of static analysis to flag variables that are defined but haven't been used.  We'll do this by looking at all the `Assign` nodes to record all the variables created.  Next we'll look at the `Name` nodes to record when the variables are used.  By comparing the two lists, we can see when a variable has been created but hasn't been used.  

**Note:**  we need to be careful here to respect python's scoping rules.  We need to record if a variable is defined within a function so that we can catch cases where a variable is created in one scope and used in another scope. To make life (and the code) simpler, we'll ignore class definitions entirely but handling that would be analagous to handling variables within a function.  Also, this misses variables used in code that is evaluated at runtime, but if you're doing that, you should be prepared for this.

### Static Analysis

Now we'll look at using static analysis to modify ASTs and possibly optimize our code.  We'll do two different optimizations:

* dead code elimination
* function inlining

#### Dead code elimination

Dead code elimination involves analyzing our code and removing code that we can guarantee will never run.  For example, in the following code: 

```
  if True:
    a = 5
  else:
    a = 10
```

we know that the ```a=5``` branch will always be taken.  So we can effectively replace a comparison and variable assignment with a single assignment.  This is potentially a next bonus by reducing our code size, and avoiding a comparison and jump.

This becomes even more powerful if we use constant folding and constant propagation to evaluate code at runtime. E.g. replacing code like

```
x = 5
y = x + 10
if y > 5:
  x = 20
```

with 

```
x = 5
y = 15
if 15 > 5:
  x = 20
```
then
```
x = 5
y = 15
x = 20
```

and finally 

```
y = 15
x = 20
```

Notice that in optimizing the code, running some optimizations like dead code elimination at different stages might be useful.

#### Function inlining

Function inlining is a bit more ambiguous optimization.  Function inlining involves taking the body of a function, and replacing function calls with the body of the function.  E.g. `a = func(a)` gets replaced with the code for `func` and instead of having `func` return a value, it is assigned to `a`.  Depending on the size of the function, this may be a net benefit or may even slow down the code.  This also is a bit more complicated since we need to propagate function arguments into the body and make sure that variables declared within the function are renamed so that they don't overwrite variables where the function was called.  

To make the code much simpler, we'll just inline functions that consist of calls to other functions. 